In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [2]:
class_num = 13
batch_size=32
target_size=(200,200)

gen = ImageDataGenerator(rescale=1/255.)
train_gen = gen.flow_from_directory('./train/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
valid_gen = gen.flow_from_directory('./valid/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)
test_gen = gen.flow_from_directory('./test/', target_size=target_size, class_mode='categorical', batch_size=batch_size, shuffle=True)

Found 39780 images belonging to 13 classes.
Found 13260 images belonging to 13 classes.
Found 13260 images belonging to 13 classes.


In [3]:
InceptionResNetV2_base_loaded = tf.keras.models.load_model('./save_models/InceptionResNetV2_13class')
EffNetV2M_base_loaded = tf.keras.models.load_model('./save_models/EffNetV2M_13class')
VGG16_base_loaded = tf.keras.models.load_model('./save_models/VGG16_13class')   

In [4]:
modelacc_InceptionResNetV2 = 0.8982
modelacc_EffNetV2M = 0.9431
modelacc_VGG16 = 0.8919

## Simple Voting

In [5]:
predcount = 0
hitcount = 0
forcount = 0
accuracy = 0

for image in test_gen:
    forcount += 1
    # predlist = InceptionResNetV2_base_loaded.predict(image)
    
    pred_InceptionResNetV2 = InceptionResNetV2_base_loaded.predict(image)
    pred_EffNetV2M = EffNetV2M_base_loaded.predict(image)
    pred_VGG16 = VGG16_base_loaded.predict(image)
    
    for i in range(len(pred_InceptionResNetV2)):
        # if np.argmax(predlist[i]) == np.argmax(image[1][i]) : hitcount += 1
        # predcount += 1
        
        votelist = [0 for x in range(class_num)]
        votelist[np.argmax(pred_InceptionResNetV2[i])] += 1*modelacc_InceptionResNetV2
        votelist[np.argmax(pred_EffNetV2M[i])] += 1*modelacc_EffNetV2M
        votelist[np.argmax(pred_VGG16[i])] += 1*modelacc_VGG16
        
        if np.argmax(votelist) == np.argmax(image[1][i]) : hitcount += 1
        predcount += 1        
        
        
        
    if forcount%20 == 0 : print(forcount, ':', hitcount / predcount * 100)
    if forcount%100 == 0 : break
accuracy = hitcount / predcount * 100
print(f'Simple Voting Accuracy : {accuracy}%')

20 : 93.59375
40 : 93.75
60 : 93.95833333333333
80 : 94.140625
100 : 94.03125
Simple Voting Accuracy : 94.03125%


## Probability Voting

In [20]:
predcount = 0
hitcount = 0
forcount = 0
accuracy = 0

for image in test_gen:
    forcount += 1
    # predlist = InceptionResNetV2_base_loaded.predict(image)
    
    pred_InceptionResNetV2 = InceptionResNetV2_base_loaded.predict(image)
    pred_EffNetV2M = EffNetV2M_base_loaded.predict(image)
    pred_VGG16 = VGG16_base_loaded.predict(image)
    
    for i in range(len(pred_InceptionResNetV2)):
        # if np.argmax(predlist[i]) == np.argmax(image[1][i]) : hitcount += 1
        # predcount += 1
        
        pred_InceptionResNetV2_accrate = np.multiply(pred_InceptionResNetV2,np.array(modelacc_InceptionResNetV2))
        pred_EffNetV2M_accrate = np.multiply(pred_EffNetV2M,np.array(modelacc_EffNetV2M))
        pred_VGG16_accrate = np.multiply(pred_VGG16,np.array(modelacc_VGG16))
        
        TEMP = [pred_InceptionResNetV2_accrate, pred_EffNetV2M_accrate, pred_VGG16_accrate]
        pred_sum = np.array(TEMP).sum(axis=0)
        
        if np.argmax(pred_sum[i]) == np.argmax(image[1][i]) : hitcount += 1
        predcount += 1        
        
    if forcount%20 == 0 : print(forcount, ':', hitcount / predcount * 100)
    if forcount%100 == 0 : break
accuracy = hitcount / predcount * 100
print(f'Probability Voting Accuracy : {accuracy}%')

20 : 95.625
40 : 95.390625
60 : 95.20833333333333
80 : 94.8828125
100 : 94.71875
Probability Voting Accuracy : 94.71875%
